In [1]:
from google.colab import drive
drive.mount('/content/drive')
dataPath ="/content/drive/MyDrive/Colab Notebooks/dataset/"

Mounted at /content/drive


*1*. 필요 라이브러리 설치

In [ ]:
### accelerate : hugging face 학습루프 가속화 라이브러리
### peft: LoRA, Prefix Tuning, P-Tuing, Prompt Tuning 과 같은 기법들을 쉽게 사용하도록 나온 라이브러리
### bitsandbytes: gpu 에서 모델을 손쉽게 압축할 수 있는 라이브러리
### trl: TRL (Transformer Reinforcement Learning) 은 transfomer언어 모델의 훈련을 위한 풀스택 라이브러리

In [2]:
!pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10
!pip install datasets
!pip install transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB

In [3]:
!pip install datasets transformers==4.37.2

In [4]:
import huggingface_hub
huggingface_hub.login()
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

hugging face 로그인

2. 모델 설정 (본인 hugging dataset 폴더 참고 ★ 변경포인트 본인의 학습데이터 셋을 hkcode_dataset 변수 내 선언해야함)

In [5]:
base_model = "yanolja/EEVE-Korean-Instruct-2.8B-v1.0"

dataset_kh = "kyounghyun/levware_ko_test"

new_model = "EEVE-Korean-Instruct-levware-en-2.8B-v1.0"

In [6]:
dataset = load_dataset(dataset_kh, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/27 [00:00<?, ? examples/s]

In [7]:
dataset[0]

{'text': '<s>[INST] 레브웨어의 AI 플랫폼 이름은? [/INST] 레브웨어의 AI 플랫폼 이름은 Lev.ai 입니다. </s>'}

In [8]:
dataset = load_dataset(dataset_kh, split="train")

In [9]:
# 데이터 확인
print(dataset[2])

{'text': '<s>[INST] 레브웨어의 주요 AI 모델은 몇 개입니까? [/INST] 레브웨어에는 5가지 모델이 있습니다: Medi.xm_Edue.xm_Soci.xm_Fact.xm_Farm.xm. </s>'}


**4**. 4비트 양자화 QLoRA 파인튜닝 (효율성) * 파라미터를 고정 시키고 추가데이터만 튜닝

In [10]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

6. 토크나이저 불러오기 (Hugginface에서 토크나이저를 로드하고 padding_side를 "right"로 설정하여 fp16과 관련된 문제해결)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

[링크 텍스트](https://)

```
# 코드로 형식 지정됨
```

### 7. PEFT 파라미터 (Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의

https://huggingface.co/docs/peft/conceptual_guides/lora

In [13]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

8. Training parameters

In [ ]:
### num_train_epochs 파라미터는 변경 포인트 임 학습데이터를 전체적으로 2번 학습할 경우 설정 2

In [14]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=5,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

**9**. model 파인튜닝

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [16]:
trainer.train()

Step,Training Loss
25,2.890000
50,1.397700
75,0.614100
100,0.264100


TrainOutput(global_step=120, training_loss=1.1048271536827088, metrics={'train_runtime': 222.1275, 'train_samples_per_second': 2.431, 'train_steps_per_second': 0.54, 'total_flos': 702098648692992.0, 'train_loss': 1.1048271536827088, 'epoch': 20.0})

In [17]:
  #logging.set_verbosity(logging.CRITICAL)
  prompt = "레브웨어가 보유하고 있는 AI 모델은 몇 개야?"
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:58943 for open-end generation.


<s>[INST] 레브웨어가 보유하고 있는 AI 모델은 몇 개야? [/INST] 레브웨어는 5개 모델을 보유하고 있습니다. oci_ai5.exe,  soci_a6.exe,  soci_a7.exe,  soci_a8.exe,  soci_a9.exe. oci_a10.exe. oci_a11.exe. 


In [18]:
logging.set_verbosity(logging.CRITICAL)

prompt = "레브웨어의 급여 처방 추천 모델에 대해 설명해줘"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어의 급여 처방 추천 모델에 대해 설명해줘 [/INST] 환자의 의료정보 데이터를 분석하여 청구 가능한 급여를 제안하여 환자에게 맞춤형 급여를 제공합니다. olution. 레브웨어는 환자의 의료정보 데이터를 분석하여 청구 가능한 급여를 제안하여 환자에게 맞춤형 급여를 제공합니다. olution. olution. olution. olution. olution. olution. olution. olution.


In [19]:
logging.set_verbosity(logging.CRITICAL)

prompt = "레브웨어의 CEO 이름 알려줘"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어의 CEO 이름 알려줘 [/INST] 대표님 이름은 류형규입니다. 직하는 바에 따르면, 직하는 바에 따르면, 류형규님. 직하는 바에 따르면, 류형규님. s직하는 바에 따르면, 류형규님. 직하는 바에 따르면, 류형규님. s직하는 바에 따르면, 류형


In [20]:
prompt = "레브웨어의 공정시간 예측 모델은 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어의 공정시간 예측 모델은 뭐야? [/INST] 전체 공정 시간은 생산에 필요한 단위 공정별 작업 조건, 투입 인력, 원료 조건 등을 바탕으로 예측됩니다. 직원님 성함이 어떻게 되시나요? [/S] 님님 이름은 김성봉입니다. 직원님 이름은 김종훈입니다. 직원님 이름은 김성봉입니다. s


In [21]:
prompt = "레브웨어의 학습 추천 모델에 대해 설명해줘"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print("result: ")
print(result)
print("==============================")
print(result[0]['generated_text'])

result: 
[{'generated_text': '<s>[INST] 레브웨어의 학습 추천 모델에 대해 설명해줘 [/INST] 사용자의 학습 데이터와 선호도를 분석하여 개인에게 맞춤형 학습 콘텐츠와 문제를 추천합니다. 이를 통해 학습 효율성을 극대화하고 사용자 경험을 개인화하여 차별화된 가치를 제공합니다. ) ocially responsible AI 개발업체로 선정되는 영예를 안았습니다 [/Socially responsible AI 개발업체로 선정되는 영예를 안았습니다 ] ) 빅데이터 기반'}]
<s>[INST] 레브웨어의 학습 추천 모델에 대해 설명해줘 [/INST] 사용자의 학습 데이터와 선호도를 분석하여 개인에게 맞춤형 학습 콘텐츠와 문제를 추천합니다. 이를 통해 학습 효율성을 극대화하고 사용자 경험을 개인화하여 차별화된 가치를 제공합니다. ) ocially responsible AI 개발업체로 선정되는 영예를 안았습니다 [/Socially responsible AI 개발업체로 선정되는 영예를 안았습니다 ] ) 빅데이터 기반


In [22]:
prompt = "레브웨어의 부사장 이름이 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print("result: ")
print(result)
print("==============================")
print(result[0]['generated_text'])

result: 
[{'generated_text': '<s>[INST] 레브웨어의 부사장 이름이 뭐야? [/INST] 레브웨어 부사장님의 이름은 김성봉입니다. 직원들의 학습 성취도를 향상시키기 위해 개인 맞춤형 학습 경로 조정 및 피드백을 제공합니다. 직원들의 학습 경로 조정과 피드백을 위해 데이터 바우처에 대한ysu.ai.data.xm의 이름은? [/INST]  직원들의 학습'}]
<s>[INST] 레브웨어의 부사장 이름이 뭐야? [/INST] 레브웨어 부사장님의 이름은 김성봉입니다. 직원들의 학습 성취도를 향상시키기 위해 개인 맞춤형 학습 경로 조정 및 피드백을 제공합니다. 직원들의 학습 경로 조정과 피드백을 위해 데이터 바우처에 대한ysu.ai.data.xm의 이름은? [/INST]  직원들의 학습


In [23]:
prompt = "레브웨어는 2023년에 어떤 인증과 참여를 달성했습니까?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어는 2023년에 어떤 인증과 참여를 달성했습니까? [/INST] 2023년 Lev.ai 은 GS인증 획득, SW 전문가 양성 프로그램 참여, 벤처기업으로 재인증, PCOE에 대한 SAP 파트너 인증 갱신 등을 진행하였습니다. 나스루.ai 은 2023년 PCOE에 대한 SAP 파트너 인증 갱신, 벤처기업으로 재인증, PCOE


In [24]:
prompt = "레브웨어의 Lev.ai 이란 무엇입니까?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어의 Lev.ai 이란 무엇입니까? [/INST] 레브웨어가 개발한 노코드를 기반으로 AI 개발과 운영이 통합된 데브옵스 플랫폼입니다. oci.a 레브웨어가 개발한 노코드를 기반으로 AI 개발과 운영이 통합된 데브옵스 플랫폼입니다.  soci.a. oci.a. oci.a.  oci.a.  


In [25]:
prompt = "Lev.ai가 무엇입니까?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Lev.ai가 무엇입니까? [/INST] Lev.ai는 AI 기반 인플루언서 마케팅 솔루션과 AI 지원 한의학 진단 솔루션을 개발한 AI 솔루션 제공업체입니다. 직해야 할 바를 알려주세요? [/S] 직해야 할 바를 알려주세요. 직해야 할 바를 알려주세요. 직해야 할 바를 알려주세요. 직해야 할 바를 알려주세요. s


In [26]:
prompt = "Levware는 어떤 회사입니까?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Levware는 어떤 회사입니까? [/INST] Levware는 제조업체입니다. ) 레브웨어와 파트너 관계를 맺었습니다. ) 금형 원가 추적을 위한 LevMCOS를 개발했습니다. s) 저작권 보호를 위해 LevCAM을 확보했습니다. s) 빅데이터 분석 솔루션인 LevMLab을 출시했습니다. s) oci.xm_Edue.


In [27]:
model.push_to_hub('kyounghyun/EEVE2.8B_KO_Finetune_test')

model.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kyounghyun/EEVE2.8B_KO_Finetune_test/commit/6363a2ba29c450bec063d1f2e95094b6daeef166', commit_message='Upload PhiForCausalLM', commit_description='', oid='6363a2ba29c450bec063d1f2e95094b6daeef166', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(58944, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=2560, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2560, out_feat